In [1]:
import os
import glob
from tqdm.notebook import tqdm
import sys
sys.path.append("/mnt/data/MaSIF_colab") 
sys.path.append("/mnt/data/MaSIF_colab/data_preprocessing") 
import numpy as np
import torch
from Bio.PDB import *
import collections.abc as container_abcs
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import Compose
import argparse
import shutil
from collections import abc as container_abcs
from helper import *
import shutil
from pdbparser.pdbparser import pdbparser
# Custom data loader and model
from data import ProteinPairsSurfaces, PairData, CenterPairAtoms, load_protein_pair,load_protein_npy
from data import RandomRotationPairAtoms, NormalizeChemFeatures, iface_valid_filter
from data_preprocessing.download_pdb import convert_to_npy, extractPDB
from data_preprocessing.convert_pdb2npy import load_structure_np
from model import dMaSIF
from data_iteration import iterate
import nglview as ng
import torch
from Bio.PDB.PDBParser import PDBParser
from scipy.spatial.distance import cdist
import subprocess
from embedding_extraction import get_model, create_folder, generate_descr, protonate_pdb
import glob

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [3]:
#define input
folder_path  = '/mnt/data/pdbs/pdb_fasta_200_94/'
all_files = glob.glob(os.path.join(folder_path, '*.pdb'))
working_dir = '/mnt/data'
# create folders
chains_dir = os.path.join(working_dir,'chains')
create_folder(chains_dir)
npy_dir = os.path.join(working_dir,'npys')
create_folder(npy_dir)
reduce_dir = os.path.join(working_dir,'reduce')
create_folder(reduce_dir)
pred_dir = os.path.join(working_dir,'preds')
create_folder(pred_dir)

# model parameters 
model_resolution = 0.7
patch_radius = 12 
model_path, supsampling = get_model(working_dir, model_resolution, patch_radius)

working_dir = '/mnt/data'
for target_pdb in all_files: 
    chains = ['A']   
    target_name = os.path.basename(target_pdb)[:-4]
    reduced_pdb = protonate_pdb(reduce_dir, target_pdb)
    convert_to_npy(reduced_pdb, chains_dir, npy_dir, chains)
    # Generate the embeddings
    pdb_name = "{n}_{c}_{c}".format(n=target_name, c=chains[0])
    info = generate_descr(model_path, pred_dir, pdb_name, npy_dir, patch_radius, model_resolution, supsampling)

100%|██████████| 1/1 [02:35<00:00, 155.25s/it]
